In [3]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect
import pandas as pd
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np

from io import StringIO

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")

wait = WebDriverWait(driver, 5)

# Step 1: Locate the iframe
iframe = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "iframe"))
)

# Step 2: Switch to the iframe
driver.switch_to.frame(iframe)

# Step 3: Locate all ng-select elements inside the iframe
dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")

# Step 4: Print the currently selected value from each dropdown
for i, dropdown in enumerate(dropdowns):
    try:
        label = dropdown.find_element(By.CSS_SELECTOR, ".ng-value-label").text.strip()
        print(f"Dropdown {i+1}: {label}")
    except:
        print(f"Dropdown {i+1}: No value selected or label not found.")

# Now you can interact with ng-select elements inside the iframe
driver.quit()

Dropdown 1: Open Ended
Dropdown 2: No value selected or label not found.
Dropdown 3: No value selected or label not found.
Dropdown 4: All


In [17]:
import time
import itertools
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
wait = WebDriverWait(driver, 10)


iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)


dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']
dropdown_3 = ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap', 'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic']
dropdown_4_value = 'All' 

combinations = list(itertools.product(dropdown_1, dropdown_2, dropdown_3))

def wait_for_iframe(timeout=30):
    for _ in range(timeout):
        try:
            iframe = driver.find_element(By.TAG_NAME, "iframe")
            return iframe
        except:
            time.sleep(1)
    raise Exception("Iframe not found after waiting")

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)
    
def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        arguments[0].value = arguments[1];
        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)

def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(3)  # Adjust based on actual load time

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(2)  # Give time for download to trigger

for idx, combo in enumerate(combinations):
    print(f"Combo {idx+1}: {combo}")

    dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
    select_dropdown(dropdowns[0], combo[0])  
    select_dropdown(dropdowns[1], combo[1])  
    select_dropdown(dropdowns[2], combo[2])
    select_dropdown(dropdowns[3], dropdown_4_value)  


    time.sleep(2)
driver.quit()

Combo 1: ('Open Ended', 'Equity', 'Large Cap')
Combo 2: ('Open Ended', 'Equity', 'Large & Mid Cap')
Combo 3: ('Open Ended', 'Equity', 'Flexi Cap')
Combo 4: ('Open Ended', 'Equity', 'Multi Cap')
Combo 5: ('Open Ended', 'Equity', 'Mid Cap')
Combo 6: ('Open Ended', 'Equity', 'Small Cap')
Combo 7: ('Open Ended', 'Equity', 'Value')
Combo 8: ('Open Ended', 'Equity', 'ELSS')
Combo 9: ('Open Ended', 'Equity', 'Contra')
Combo 10: ('Open Ended', 'Equity', 'Dividend Yield')
Combo 11: ('Open Ended', 'Equity', 'Focused')
Combo 12: ('Open Ended', 'Equity', 'Sectoral / Thematic')
Combo 13: ('Open Ended', 'Debt', 'Large Cap')
Combo 14: ('Open Ended', 'Debt', 'Large & Mid Cap')
Combo 15: ('Open Ended', 'Debt', 'Flexi Cap')
Combo 16: ('Open Ended', 'Debt', 'Multi Cap')
Combo 17: ('Open Ended', 'Debt', 'Mid Cap')
Combo 18: ('Open Ended', 'Debt', 'Small Cap')
Combo 19: ('Open Ended', 'Debt', 'Value')
Combo 20: ('Open Ended', 'Debt', 'ELSS')
Combo 21: ('Open Ended', 'Debt', 'Contra')
Combo 22: ('Open Ended

InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=140.0.7339.81); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7de8efca5+79861]
	GetHandleVerifier [0x0x7ff7de8efd00+79952]
	(No symbol) [0x0x7ff7de66cada]
	(No symbol) [0x0x7ff7de658995]
	(No symbol) [0x0x7ff7de67dbda]
	(No symbol) [0x0x7ff7de6f4346]
	(No symbol) [0x0x7ff7de7148c2]
	(No symbol) [0x0x7ff7de6ec8b3]
	(No symbol) [0x0x7ff7de6b5272]
	(No symbol) [0x0x7ff7de6b6043]
	GetHandleVerifier [0x0x7ff7debab9dd+2946349]
	GetHandleVerifier [0x0x7ff7deba5c5a+2922410]
	GetHandleVerifier [0x0x7ff7debc59e7+3052855]
	GetHandleVerifier [0x0x7ff7de90aa8e+189918]
	GetHandleVerifier [0x0x7ff7de912a2f+222591]
	GetHandleVerifier [0x0x7ff7de8f8ac4+116244]
	GetHandleVerifier [0x0x7ff7

In [ ]:
# import time
# import itertools
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os, time, itertools, glob
from selenium.webdriver.chrome.options import Options

download_dir = r"C:\Users\kaustubh.keny\Downloads"

chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(options=chrome_options)

# Setup
driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
wait = WebDriverWait(driver, 10)

# Dropdown values
dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']
dropdown_3 = ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap', 'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic']
dropdown_4_value = 'All'
combinations = list(itertools.product(dropdown_1, dropdown_2, dropdown_3))

# Utility functions
def wait_for_iframe(timeout=30):
    for _ in range(timeout):
        try:
            iframe = driver.find_element(By.TAG_NAME, "iframe")
            return iframe
        except:
            time.sleep(1)
    raise Exception("Iframe not found after waiting")

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        const input = arguments[0];
        const nativeInputValueSetter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, 'value').set;
        nativeInputValueSetter.call(input, arguments[1]);
        input.dispatchEvent(new Event('input', { bubbles: true }));
        input.dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)


def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(3)

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(2)

def rename_latest_excel(combo):
    time.sleep(3)  # wait for download to finish
    files = glob.glob(os.path.join(download_dir, "*.xlsx"))
    latest_file = max(files, key=os.path.getctime)

    original_name = os.path.basename(latest_file).replace(".xlsx", "")
    combo_str = "_".join(combo).replace(" ", "-")

    new_name = f"{original_name}_{combo_str}.xlsx"
    new_path = os.path.join(download_dir, new_name)

    os.rename(latest_file, new_path)
    print(f"✅ Renamed to: {new_name}")

for idx, combo in enumerate(combinations):
    try:
        print(f"🔄 Combo {idx+1}: {combo}")

        iframe = wait_for_iframe()
        driver.switch_to.frame(iframe)

        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[0], combo[0])
        select_dropdown(dropdowns[1], combo[1])
        select_dropdown(dropdowns[2], combo[2])
        select_dropdown(dropdowns[3], dropdown_4_value)

        set_date("31-Jul-2025")
        click_go()
        download_excel()
        print("📥 Download triggered — waiting for file...")
        rename_latest_excel(combo)

        driver.switch_to.default_content()

    except Exception as e:
        print(f"⚠️ Skipping combo {combo} due to error: {e}")
        driver.switch_to.default_content()
        continue

driver.quit()

🔄 Combo 1: ('Open Ended', 'Equity', 'Large Cap')
📥 Download triggered — waiting for file...
✅ Renamed to: Fund-Performance-04-Sep-2025--1856_Open-Ended_Equity_Large-Cap.xlsx
🔄 Combo 2: ('Open Ended', 'Equity', 'Large & Mid Cap')
📥 Download triggered — waiting for file...
✅ Renamed to: Fund-Performance-04-Sep-2025--1857_Open-Ended_Equity_Large-&-Mid-Cap.xlsx
🔄 Combo 3: ('Open Ended', 'Equity', 'Flexi Cap')
📥 Download triggered — waiting for file...
✅ Renamed to: Fund-Performance-04-Sep-2025--1857_Open-Ended_Equity_Flexi-Cap.xlsx
🔄 Combo 4: ('Open Ended', 'Equity', 'Multi Cap')
📥 Download triggered — waiting for file...
✅ Renamed to: Fund-Performance-04-Sep-2025--1857_Open-Ended_Equity_Multi-Cap.xlsx
🔄 Combo 5: ('Open Ended', 'Equity', 'Mid Cap')
📥 Download triggered — waiting for file...
✅ Renamed to: Fund-Performance-04-Sep-2025--1857_Open-Ended_Equity_Mid-Cap.xlsx
🔄 Combo 6: ('Open Ended', 'Equity', 'Small Cap')
📥 Download triggered — waiting for file...
✅ Renamed to: Fund-Performance

In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")

wait = WebDriverWait(driver, 10)

# Step 1: Switch to the iframe
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

# Step 2: Locate all ng-select dropdowns
dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")

# Step 3: Loop through each dropdown and collect options
all_dropdown_options = []

for i, dropdown in enumerate(dropdowns):
    try:
        # Open the dropdown
        arrow = dropdown.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
        arrow.click()
        time.sleep(1)  # Let Angular render the options

        # Wait for options to appear
        options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))

        # Collect option texts
        option_texts = [opt.text.strip() for opt in options if opt.text.strip()]
        all_dropdown_options.append({
            "dropdown_index": i + 1,
            "options": option_texts
        })

        print(f"Dropdown {i+1} options:")
        for text in option_texts:
            print(" -", text)

        # Close dropdown by clicking outside (optional)
        driver.execute_script("document.body.click()")
        time.sleep(1)

    except Exception as e:
        print(f"Dropdown {i+1}: Error collecting options → {e}")
        all_dropdown_options.append({
            "dropdown_index": i + 1,
            "options": []
        })

# Step 4: Switch back to main content (if needed)
driver.switch_to.default_content()

# Optional: Print the full collected list
print("\n✅ All collected dropdown options:")
for entry in all_dropdown_options:
    print(f"Dropdown {entry['dropdown_index']}: {entry['options']}")
    
    
driver.quit()

Dropdown 1 options:
 - Open Ended
 - Close Ended
Dropdown 2 options:
 - Equity
 - Debt
 - Hybrid
 - Solution Oriented
 - Other
Dropdown 3 options:
 - Large Cap
 - Large & Mid Cap
 - Flexi Cap
 - Multi Cap
 - Mid Cap
 - Small Cap
 - Value
 - ELSS
 - Contra
 - Dividend Yield
 - Focused
 - Sectoral / Thematic
Dropdown 4 options:
 - All
 - Jio BlackRock Mutual Fund
 - 360 ONE Mutual Fund
 - Aditya Birla Sun Life Mutual Fund
 - Angel One Mutual Fund
 - Axis Mutual Fund
 - Bajaj Finserv Mutual Fund
 - Bandhan Mutual Fund
 - Bank of India Mutual Fund
 - Baroda BNP Paribas Mutual Fund
 - Canara Robeco Mutual Fund
 - DSP Mutual Fund
 - Edelweiss Mutual Fund
 - Franklin Templeton Mutual Fund
 - Groww Mutual Fund
 - HDFC Mutual Fund
 - Helios Mutual Fund
 - HSBC Mutual Fund
 - ICICI Prudential Mutual Fund
 - Invesco Mutual Fund
 - ITI Mutual Fund
 - JM Financial Mutual Fund
 - Kotak Mahindra Mutual Fund
 - LIC Mutual Fund
 - Mahindra Manulife Mutual Fund
 - Mirae Asset Mutual Fund
 - Motilal Oswa

In [ ]:
path = "https://www.jkbank.com/interest-rates"
options = Options()
driver = webdriver.Chrome(options=options)

driver.get(path)
tabList = driver.find_elements(By.CSS_SELECTOR,".owl-nav-carousel li a")
print(f"Total tabs found: {len(tabList)}")
for tab in tabList:
    try:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(1) 

        ActionChains(driver).move_to_element(tab).perform()

        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
        print(tab.get_attribute("innerText"))
        pprint.pprint(tab.get_attribute("outerHTML"))
        tab.click()
        time.sleep(1)
        
        pprint.pprint(tab.get_attribute("outerHTML"))
        print("==============================")
        
        # tables = driver.find_elements(By.TAG_NAME, "table")
        # for idx,table in enumerate(tables):
        #     content = table.get_attribute("outerHTML")
        #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
        #         f.write(content)
    except Exception as e:
        print(f"Failed to click tab: {e}")
    
driver.quit()

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
pathList = [r"25-09-03T16-21-30_cache.json",r"25-09-04T16-02-03_cache.json"]

for path in pathList:
    
    root_file = os.path.basename(path).split("_")[0]
    function_to_execute = {
    "primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
    "secondary":[["sha1","norm_table","SHA_ONE"]],
    }
    try:
        data = Helper.load_json(path, typ="json5")
        processed_data = ops.runner(data,function_to_execute)
        Helper.save_json(processed_data,f"{root_file}_process.json")
    except Exception as e:
        print(f"\nError in Processing.. Skipping, {e}")
        continue

In [2]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()
yesterday = Helper.load_json("25-09-03T16-21-30_process.json")
today = Helper.load_json("25-09-04T16-02-03_process.json")

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-030925.json")

ops.generate_sorted_excel_report(result)

'DepositRate_Comparison_Report.xlsx'

In [ ]:
"Indian Bank"

link1 = "https://www.indianbank.in/departments/fcnr-b-deposits/"
ink2 = "https://www.indianbank.in/departments/interest-rates-for-deposits-of-above-rs-5-00crore/"

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import random, time, math, ssl

ssl._create_default_https_context = ssl._create_stdlib_context

def human_scroll(driver, total_scroll=2000, step=50):
    for y in range(0, total_scroll, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.05, 0.3))

def human_move_mouse(driver, start=(0, 0), end=(300, 300), steps=30):
    actions = ActionChains(driver)
    x1, y1 = start
    x2, y2 = end
    for t in range(steps):
        progress = t / steps
        x = x1 + (x2 - x1) * progress + random.uniform(-2, 2)
        y = y1 + (y2 - y1) * progress + math.sin(progress * math.pi) * 20
        actions.move_by_offset(int(x), int(y))
    actions.perform()

def human_click(elem):
    time.sleep(random.uniform(0.2, 1.5))
    elem.click()

profile_path = Helper.load_json("paths.json").get("profile_path")
profile_dir = Helper.load_json("paths.json").get("profile_name")

options = Options()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

human_scroll(driver, total_scroll=1500, step=70)
human_move_mouse(driver, start=(0, 0), end=(200, 200), steps=40)

try:
    table = driver.find_element(By.TAG_NAME, "table")
    human_click(table) # Just demo click
    print("Table found:", table.get_attribute("outerHTML")[:200])
except Exception as e:
    print("No table found:", e)

time.sleep(5)
driver.quit()


In [ ]:
#   GENERIC_ACTION_PACKETS: {
#     action_table: {
#       action: "table",
#       by: "css",
#       value: "table",
#       wait_until: "present",
#       multiple: true,
#       skip_if_not_found: true,
#       default_wait: 10,
#       timeout: 30,
#       log_message: "Save Html Tables.",
#     },
#     action_screenshot: {
#       action: "screenshot",
#       by: "css",
#       value: ".main-content",
#       wait_until: "visible",
#       skip_if_not_found: true,
#       timeout: 20,
#       log_message: "Screen Shotted the Web Page.",
#     },
    # action_download: {
    #   action: "download",
    #   by: "css",
    #   value: "a.FDlink.pdf-data",
    #   multiple: true,
    #   wait_until: "clickable",
    #   default_wait: 5,
    #   timeout: 20,
    #   log_message: "Axis Bank",
    # },
#     action_website: {
#       action: "website",
#       by: "css",
#       value: "body",
#       url: "https://bankofindia.co.in/interest-rate/saving-bank-deposit-rate",
#       default_wait: 5,
#     },
#     action_html: {
#       action: "html",
#       by: "css",
#       value: "body",
#       wait_until: "present",
#       // multiple: true,
#       skip_if_not_found: true,
#       default_wait: 10,
#       timeout: 30,
#       log_message: "Save Html.",
#     },
#   },

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

path = "https://www.nseindia.com/market-data/equity-derivatives-watch"
options = Options()
driver = webdriver.Chrome(options=options)
driver.get(path)

time.sleep(5)

select_element = driver.find_element(By.CSS_SELECTOR, "select.no-border-radius")
select = Select(select_element)

for option in select.options:
    try:
        value = option.get_attribute("value")
        print(f"Selecting: {option.text} ({value})")
        
        select.select_by_value(value)
        time.sleep(2)

        tables = driver.find_elements(By.CSS_SELECTOR, "#eqderivativesTable")
        print(f"Found {len(tables)} tables for {option.text}")
        
        for idx, table in enumerate(tables):
            content = table.get_attribute("outerHTML")
            with open(f"{value}_table_{idx}.html", "w+", encoding="utf-8") as f:
                f.write(content)

    except Exception as e:
        print(f"Failed to select {option.text}: {e}")from app.operation_executor import OperationExecutor

# path = "https://www.jkbank.com/interest-rates"
# options = Options()
# driver = webdriver.Chrome(options=options)

# driver.get(path)
# tabList = driver.find_elements(By.CSS_SELECTOR,".owl-nav-carousel li a")
# print(f"Total tabs found: {len(tabList)}")
# for tab in tabList:
#     try:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
#         time.sleep(1) 

#         ActionChains(driver).move_to_element(tab).perform()

#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
#         print(tab.get_attribute("innerText"))
#         tab.click()
#         time.sleep(1)
        
#         tables = driver.find_elements(By.TAG_NAME, "table")
#         # for idx,table in enumerate(tables):
#         #     content = table.get_attribute("outerHTML")
#         #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
#         #         f.write(content)
#     except Exception as e:
#         print(f"Failed to click tab: {e}")
        
        
# path = "https://www.icicibank.com/personal-banking/deposits/fixed-deposit/fd-interest-rates?ITM=nli_cms_FD_rates_menu_navigation"
# options = Options()
# driver = webdriver.Chrome(options=options)

# driver.get(path)
# tabList = driver.find_elements(By.CSS_SELECTOR,".tabs-list button")
# print(f"Total tabs found: {len(tabList)}")
# for tab in tabList:
#     try:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
#         time.sleep(1) 

#         ActionChains(driver).move_to_element(tab).perform()

#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
#         print(tab.get_attribute("innerText"))
#         tab.click()
#         time.sleep(1)
        
#         tables = driver.find_elements(By.TAG_NAME, "table")
#         # for idx,table in enumerate(tables):
#         #     content = table.get_attribute("outerHTML")
#         #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
#         #         f.write(content)
#     except Exception as e:
#         print(f"Failed to click tab: {e}")
